In [299]:
from ipynb.fs.full.utils import read_test_input
from itertools import combinations, permutations

In [300]:
filename = "day7_input.txt"

In [309]:
read_input = read_test_input(filename)

In [310]:
read_input = read_input[0].split(',')

In [311]:
read_input = [int(val) for val in read_input]

In [312]:
def get_value(total_input, param, mode):
    if mode == 0:
        return total_input[param]
    elif mode == 1:
        return param
    else:
        raise Exception("invalid mode")

In [313]:
def add(total_input, pointer, *args, input=None):
#     print("add")
    param_value1, param_value2, output = args # these ar 2-tuples
    result = get_value(total_input, *param_value1) + get_value(total_input, *param_value2)
    output_position = output[0]
    total_input[output_position] = result
    return None, pointer + len(args) + 1, input is not None

In [314]:
def multiply(total_input, pointer, *args,input=None):
#     print("mul")
    param_value1, param_value2, output = args # these ar 2-tuples
    result = get_value(total_input, *param_value1) * get_value(total_input, *param_value2)
    output_position = output[0]
    total_input[output_position] = result
    return None, pointer + len(args) + 1, input is not None

In [315]:
def store(total_input, pointer, pos, input=None):
#     print("store")
    output_position = pos[0]
    total_input[output_position] = input
    return None, pointer + 1 + 1, input is not None

In [316]:
def output(total_input, pointer, pos, input=None):
#     print("out")
    return get_value(total_input, *pos), pointer + 1 + 1, input is not None

In [317]:
def jump_if_true(total_input, pointer, *args, input=None):
#     print("jit")
    arg1, arg2 = args
    if get_value(total_input, *arg1) != 0:
        pointer = get_value(total_input, *arg2)
    else:
        pointer += len(args) + 1
    return "jit", pointer, input is not None

In [318]:
def jump_if_false(total_input, pointer, *args, input=None):
#     print("jif")
    arg1, arg2 = args
    if get_value(total_input, *arg1) == 0:
        pointer = get_value(total_input, *arg2)
    else:
        pointer += len(args) + 1
    return "jif", pointer, input is not None

In [319]:
def less_than(total_input, pointer, *args, input=None):
#     print("lt")
    arg1, arg2, pos = args
    if get_value(total_input, *arg1) < get_value(total_input, *arg2):
        total_input[pos[0]] = 1
    else:
        total_input[pos[0]] = 0
    return "lt", pointer + len(args) + 1, input is not None

In [320]:
def equals(total_input, pointer, *args, input=None):
#     print("eq")
    arg1, arg2, pos = args
    if get_value(total_input, *arg1) == get_value(total_input, *arg2):
        total_input[pos[0]] = 1
    else:
        total_input[pos[0]] = 0
    return "eq", pointer + len(args) + 1, input is not None

In [321]:
def exit(total_input, pointer, input=None):
#     print("ex")
    return "Exit", pointer + 1, input is not None

In [322]:
INSTRUCTIONS = {
    1: {"fn": add, "number_of_parameters": 3},
    2: {"fn": multiply, "number_of_parameters": 3},
    3: {"fn": store, "number_of_parameters": 1},
    4: {"fn": output, "number_of_parameters": 1},
    5: {"fn": jump_if_true, "number_of_parameters": 2},
    6: {"fn": jump_if_false, "number_of_parameters": 2},
    7: {"fn": less_than, "number_of_parameters": 3},
    8: {"fn": equals, "number_of_parameters": 3},
    99: {"fn": exit, "number_of_parameters": 0},
}

In [323]:
def read_opcode(opcode: str):
    decoded_opcode = {}
    decoded_opcode['instruction'] = int(opcode[-2:])
    parameter_mode_str = opcode[:-2]
    parameter_modes = []
    for i in range(1, 4):
        try:
            parameter_modes.append(int(parameter_mode_str[-i]))
        except IndexError:
            parameter_modes.append(0)
    decoded_opcode["parameter_modes"] = parameter_modes
    return decoded_opcode

In [324]:
def run_program(total_input, input_val1, input_val2, pointer=0, input_val_used=False, result=None):
    
    if result is not None and isinstance(result, int):
        return result
    elif result == "Exit":
        return result
    if input_val_used:
        input_val = input_val2
    else:
        input_val = input_val1
    if pointer < len(total_input):
#         print(f"Pointer {pointer}")
        # find opcode at pointer
        opcode = str(total_input[pointer]).zfill(5)
        decoded_opcode = read_opcode(opcode)
#         print(decoded_opcode)
#         print(input_val)
#         print((opcode, decoded_opcode))
        instructions = INSTRUCTIONS[decoded_opcode["instruction"]]

        first_param_pointer = pointer + 1
        parameters = total_input[first_param_pointer: first_param_pointer + instructions["number_of_parameters"]]
        parameters_and_modes = list(zip(parameters, decoded_opcode['parameter_modes']))
#         print(parameters_and_modes)
                                   
        result, pointer_position, input_val_used = instructions["fn"](total_input, pointer, *parameters_and_modes, input=input_val)
#         print(result)
#         print(total_input)
        return run_program(total_input, input_val1, input_val2, pointer_position, input_val_used, result)

    

In [325]:
class Amp:
    
    def __init__(self, program):
        self.program = list(program)
        self.done = False
        self.pointer = 0
        self.input = []
        self.output = []
    
    def add_input(self, value):
        self.input.append(value)
        self.run_program()
        
    def run_program(self):
        while True:
            # find opcode at pointer
            opcode = str(self.program[self.pointer]).zfill(5)
            decoded_opcode = read_opcode(opcode)
            instructions = INSTRUCTIONS[decoded_opcode["instruction"]]
            if opcode == "00099":
                self.done = True
                return
            if opcode == "00003" and len(self.input) == 0:
                return
            first_param_pointer = self.pointer + 1
            parameters = self.program[first_param_pointer: first_param_pointer + instructions["number_of_parameters"]]
            parameters_and_modes = list(zip(parameters, decoded_opcode['parameter_modes']))
            
            result, new_pointer, in_val_used= instructions["fn"](self.program, self.pointer, *parameters_and_modes, input=self.input[0] if self.input else None)
            self.pointer = new_pointer
            if in_val_used:
                self.input.pop(0)
            if result is not None and isinstance(result, int):
                self.output.append(result)
                        

In [328]:
def run_sequence(program, phase_sequence):
    amps = [Amp(program) for phase in phase_sequence]
    for phase, amp in zip(phase_sequence, amps):
        amp.add_input(phase)
    amps[0].add_input(0)

    while amps[4].done is False:
        for i, amp in enumerate(amps):
            amps[(i+1) % 5].add_input(amp.output[-1])

    return amps[4].output[-1]

In [329]:
max(run_sequence(read_input, sequence) for sequence in permutations([5,6,7,8,9], 5))

21844737